## Load the meta data into a dataframe

In [ ]:
import pandas as pd

df = pd.read_excel('../data/meta_data.xlsx', sheet_name='AllAnimals')

df


## Plot the locations of the camera traps on a map 

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
import numpy as np

# Convert 'na' strings to NaN and drop rows with NaN in Longitude or Latitude
df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')
df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
df_clean = df.dropna(subset=['Longitude', 'Latitude'])

# Select unique locations based on Longitude and Latitude
unique_locations = df_clean[['Longitude', 'Latitude']].drop_duplicates()

# Create a GeoDataFrame, specifying the coordinate reference system (CRS) as WGS84 (epsg:4326)
gdf = gpd.GeoDataFrame(unique_locations, geometry=gpd.points_from_xy(unique_locations.Longitude, unique_locations.Latitude))
gdf.crs = "EPSG:4326"

# Convert the GeoDataFrame to Web Mercator projection (epsg:3857) for use with contextily
gdf = gdf.to_crs(epsg=3857)

# Plotting
fig, ax = plt.subplots(figsize=(10, 10))
gdf.plot(ax=ax, color='red', markersize=5)
ctx.add_basemap(ax, source=ctx.providers.OpenTopoMap)
ax.set_axis_off()
plt.show()


## Plot a bar char of the count for each species

In [ ]:
# Count the occurrences of each species
species_counts = df['Species'].value_counts()

# Plot the counts as a bar chart
plt.figure(figsize=(10, 8))  # Adjust the size as needed
ax = species_counts.plot(kind='bar')
plt.title('Frequency of Each Species')
plt.xlabel('Species')
plt.ylabel('Frequency')
plt.xticks(rotation=45, ha="right")  # Rotate species names for better readability
plt.tight_layout()  # Adjust layout to make room for the rotated x-axis labels

# Add count labels to the top of the bars
for i, v in enumerate(species_counts):
    ax.text(i, v, str(v), ha='center', va='bottom')

plt.show()


## Plot the frequency of detections of wild boar on a map

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx

# Filter for "Wild boar" species
wild_boar_df = df[df['Species'] == 'Wild boar']

# Count detections at each unique location
location_counts = wild_boar_df.groupby(['Longitude', 'Latitude']).size().reset_index(name='Counts')

# Create a GeoDataFrame from the filtered data
gdf = gpd.GeoDataFrame(location_counts, geometry=gpd.points_from_xy(location_counts.Longitude, location_counts.Latitude))

# Set the coordinate reference system (CRS) to WGS84
gdf.crs = 'EPSG:4326'

# Convert the GeoDataFrame to Web Mercator projection for use with contextily
gdf = gdf.to_crs(epsg=3857)

# Plotting
fig, ax = plt.subplots(figsize=(10, 10))

# Adjust marker size based on the count of detections. The 'Counts' value is scaled to ensure visibility on the map.
gdf.plot(ax=ax, markersize=gdf['Counts']*10, color='red', alpha=0.5)

# Add a basemap
ctx.add_basemap(ax, source=ctx.providers.OpenTopoMap)

# Remove axis for a cleaner look
ax.set_axis_off()

plt.show()